In [ ]:
# GAN for training loop from scratch

In [ ]:
#training the discriminator
#lấy một batch từ các điểm ngẫu nhiên trong 
#tạo ảnh giả từ các điểm đã chọn qua generator model
#lấy các hình ảnh thực và kết hợp chúng thành hình ảnh đã tạo
#so sánh model discriminator từ các ảnh thực và loại đã tạo ra

In [ ]:
#traning the generator
#lấy điểm ngẫu nhiên 
#tạo hình ảnh từ điểm ngẫu nhiên thông qua generator
#đối chiếu các hình ảnh thực và hình ảnh được tạo ra qua generator
#training model generator để phân biệt

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import time

In [10]:
#tạo discriminator phân biệt số giả và số thật
discriminator = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)
discriminator.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 64)        640       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
global_max_pooling2d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 74,625
Trainable params: 74,625
Non-trainable params: 0
_________________________________________________

In [11]:
#tạo generator, đầu ra có kích thước (28*28*1)
latent_dim = 128

generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        # We want to generate 128 coefficients to reshape into a 7x7x128 map
        #
        layers.Dense(7 * 7 * 128),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, 128)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

In [ ]:
#creat training loop


In [12]:
# Instantiate one optimizer for the discriminator and another for the generator.
#khởi tạo 2 optimizer cho discriminator và generator
d_optimizer = keras.optimizers.Adam(learning_rate=0.0003)
g_optimizer = keras.optimizers.Adam(learning_rate=0.0004)

# Instantiate a loss function.
#khởi tạo 1 loss function
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)


@tf.function
def train_step(real_images):
    # Sample random points in the latent space
    #tạo các điểm ngẫu nhiên trong latent space
    random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
    # Decode them to fake images
    #mã hóa tạo thành ảnh ảo
    generated_images = generator(random_latent_vectors)
    # Combine them with real images
    # phối hợp cùng ảnh thật
    combined_images = tf.concat([generated_images, real_images], axis=0)

    # Assemble labels discriminating real from fake images
    # Láp nhãn của ảnh trong discriminator và ảnh giả
    labels = tf.concat(
        [tf.ones((batch_size, 1)), tf.zeros((real_images.shape[0], 1))], axis=0
    )
    # Add random noise to the labels - important trick!
    #thêm nhiễu ngẫu nhiên nhãn
    labels += 0.05 * tf.random.uniform(labels.shape)

    # Train the discriminator
    #train trên mạng discriminator
    with tf.GradientTape() as tape:
        predictions = discriminator(combined_images)
        d_loss = loss_fn(labels, predictions)
    grads = tape.gradient(d_loss, discriminator.trainable_weights)
    d_optimizer.apply_gradients(zip(grads, discriminator.trainable_weights))

    # Sample random points in the latent space
    #tạo điểm ngẫu nhiên trên latent space
    random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
    # Assemble labels that say "all real images"
    misleading_labels = tf.zeros((batch_size, 1))

    # Train the generator (note that we should *not* update the weights
    # of the discriminator)!
    #train the generator ( chú ý! không nên update trọng số của discriminator)
    with tf.GradientTape() as tape:
        predictions = discriminator(generator(random_latent_vectors))
        g_loss = loss_fn(misleading_labels, predictions)
    grads = tape.gradient(g_loss, generator.trainable_weights)
    g_optimizer.apply_gradients(zip(grads, generator.trainable_weights))
    return d_loss, g_loss, generated_images

In [30]:
import os

        
# Prepare the dataset. We use both the training & test MNIST digits.
#dữ liệu mnist 
batch_size = 64
(x_train, _), (x_test, _) = keras.datasets.fashion_mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_digits = all_digits.astype("float32") / 255.0
all_digits = np.reshape(all_digits, (-1, 28, 28, 1))
dataset = tf.data.Dataset.from_tensor_slices(all_digits)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

epochs = 2  # In practice you need at least 20 epochs to generate nice digits.
save_dir = "./" #thực tế, cần ít nhất 20 epochs để tạo ra chữ ký đẹp

for epoch in range(epochs):
    print("\nStart epoch", epoch)
    start_time = time.time()
    for step, real_images in enumerate(dataset):
        # Train the discriminator & generator on one batch of real images.
        #train 2 mạng discriminator và generator cùng 1 batch trong ảnh thật
        d_loss, g_loss, generated_images = train_step(real_images)

        # Logging.
        if step % 200 == 0:
            # Print metrics
            print("discriminator loss at step %d: %.2f" % (step, d_loss))
            print("adversarial loss at step %d: %.2f" % (step, g_loss))

            # Save one generated image
            img = tf.keras.preprocessing.image.array_to_img(
                generated_images[0] * 255.0, scale=False
            )
            img.save(os.path.join(save_dir, "generated_img" + str(step) + ".png"))
        
        # To limit execution time we stop after 10 steps.
        # Remove the lines below to actually train the model!
        print("Time taken: %.2fs" % (time.time() - start_time))
            if step > 10:
                break
       
        

IndentationError: unexpected indent (<ipython-input-30-2dc5a43e4870>, line 40)